<a href="https://colab.research.google.com/github/Firojpaudel/GenAI-Chronicles/blob/main/BERTs/BERT_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Trying to Implement BERT**: _Using Huggingface_ 🤗

---

In [ ]:
##@ First lets install the huggingface trannsformers, datasets, evaluate and accelerate
! pip install transformers datasets evaluate accelerate